In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.io import read_image
import os

# Define the neural network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer with 3 input channels (RGB), 16 output channels, 3x3 kernel, and padding of 1
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # Max pooling layer with a 2x2 kernel and stride 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Second convolutional layer with 16 input channels, 32 output channels, 3x3 kernel, and padding of 1
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Fully connected layer with an input size based on the assumed image size (32x32x32) and 2 output classes
        self.fc1 = nn.Linear(32 * 32 * 32, 2)

    def forward(self, x):
        # First convolutional layer followed by ReLU activation and max pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Second convolutional layer followed by ReLU activation and max pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Flatten the output before passing through the fully connected layer
        x = x.view(-1, 32 * 32 * 32)
        # Fully connected layer
        x = self.fc1(x)
        return x

# Load the dataset
data_transform = transforms.Compose([
    # Resize the input images to 128x128 pixels
    transforms.Resize((128, 128)),
    # Convert images to PyTorch tensors
    transforms.ToTensor()
])

# Load the dataset from the specified root folder and apply data transformations
dataset = datasets.ImageFolder(root='F:/PROJECT/Machine Learning/Capstone/datasets/', transform=data_transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoader instances for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
# Cross-entropy loss function for classification problems
criterion = nn.CrossEntropyLoss()
# Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    for images, labels in train_loader:
        # Zero the gradients to avoid accumulating them
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        # Calculate the loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Update the model parameters
        optimizer.step()

    # Print the loss at the end of each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Evaluation
# Set the model to evaluation mode (no gradient computation)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # Forward pass
        outputs = model(images)
        # Get the predicted class for each sample
        _, predicted = torch.max(outputs.data, 1)
        # Update the total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print the test accuracy
accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy}%')
